# Hyperparameter Tuning

Improting Libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             roc_auc_score, average_precision_score, roc_curve, precision_recall_curve)
from lifelines import KaplanMeierFitter, CoxPHFitter

from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

from skorch import NeuralNetClassifier

from scipy.special import expit

from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt

In [ ]:
torch.cuda.is_available()

Defining Focal Loss

In [ ]:
class FocalLoss(nn.Module):
    """
    Binary focal loss for logits.
    """
    def __init__(self, alpha=1.0, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, logits, targets):
        targets = targets.view(-1, 1).type_as(logits)  # Make sure targets are (batch, 1) and same type as logits
        bce_loss = nn.functional.binary_cross_entropy_with_logits(
            logits, targets, reduction='none'
        )

        # Compute pt
        pt = torch.exp(-bce_loss)

        # Apply focal term
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss


Defining Model Architecture

In [ ]:
class DeepBinary(nn.Module):
    """
    Deep feedforward model for binary classification (outputs logits).
    """
    def __init__(self, input_dim, hidden_dim=64, num_layers=4, dropout_rate=0.25):
        super().__init__()
        layers = []
        in_dim = input_dim

        for _ in range(num_layers):
            layers.append(nn.Linear(in_dim, hidden_dim))
            layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            in_dim = hidden_dim

        # Final output layer — raw logits (no sigmoid here!)
        layers.append(nn.Linear(hidden_dim, 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)  # raw logits


Neural Net Binary Classifier for Skorch

In [ ]:
class NeuralNetBinaryClassifier(NeuralNetClassifier):
    """
    Skorch-compatible wrapper for binary classification that ensures predict_proba works.
    """
    def predict_proba(self, X):
        # Get logits
        logits = self.forward(X).detach().cpu().numpy()
        # Apply sigmoid for probabilities
        probs = expit(logits)
        # Return shape (n_samples, 2) for sklearn compatibility
        return np.hstack((1 - probs, probs))

Loading in Data

In [ ]:
data = pd.read_csv(
    f"https://raw.githubusercontent.com/JackWJW/LGG_Prognosis_Prediction/main/Tidied_Datasets/tidied_integrated_df_13.csv"
).drop(columns=["Unnamed: 0"])

dss_info = data[["DSS", "DSS.time"]]

X = data.drop(columns=["Srv", "DSS", "DSS.time"])
y = LabelEncoder().fit_transform(data["Srv"])

X_train = X
y_train = y

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

smote_oversampler = SMOTE()
X_train_scaled, y_train = smote_oversampler.fit_resample(X_train_scaled,y_train)

Defining Model Search Spaces

In [ ]:
search_spaces = {
    "SVM": (
        SVC(probability=True, class_weight="balanced", kernel = 'rbf', gamma='scale', random_state=42),
        {
            "C": Real(0.01, 0.1)
        }
    ),
    "RandomForest": (
        RandomForestClassifier(class_weight="balanced", random_state=42),
        {
            "n_estimators": Integer(50, 500),
            "max_depth": Integer(2, 20),
            "min_samples_split": Integer(2, 20)
        }
    ),
    "XGBoost": (
        XGBClassifier(eval_metric="logloss", random_state=42),
        {
            "n_estimators": Integer(50, 500),
            "max_depth": Integer(2, 10),
            "learning_rate": Real(0.01, 0.3, prior="log-uniform"),
            "subsample": Real(0.5, 1.0)
        }
    ),
    "LogisticRegression": (
        LogisticRegression(max_iter=5000, class_weight="balanced", solver="lbfgs"),
        {
            "C": Real(0.01, 100, prior="log-uniform")
        }
    ),
    "NaiveBayes": (
        GaussianNB(),
        {
            "var_smoothing": Real(1e-10, 1e-6, prior="log-uniform")
        }
    )
}


net = NeuralNetBinaryClassifier(
    module=DeepBinary,
    module__input_dim=X_train.shape[1],
    criterion=FocalLoss,
    criterion__alpha=0.25,
    criterion__gamma=2.0,
    max_epochs=250,
    lr=1e-3,
    optimizer=torch.optim.Adam,
    batch_size=128,
    iterator_train__shuffle=True,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    verbose=0
)


deep_search_space = {
    "lr": Real(1e-4, 1e-1, prior="log-uniform"),
    "module__hidden_dim": Integer(16, 1024),
    "module__num_layers": Integer(2, 8),
    "module__dropout_rate": Real(0.0, 0.75),
    "criterion__alpha": Real(0.0, 0.5),
    "criterion__gamma": Real(2.0, 5.0)
}


Performing Optimisation

In [ ]:
best_params_df = pd.DataFrame()
trained_models = {}
val_probs = {}

for model_name, (model, space) in search_spaces.items():
    print(f"\n Optimizing {model_name}...")
    opt = BayesSearchCV(
        model,
        space,
        n_iter=250,
        cv=5,
        scoring='f1_macro',
        n_jobs=-1,
        random_state=42,
        optimizer_kwargs={'n_initial_points':50}
    )
    opt.fit(X_train_scaled, y_train)
    trained_models[model_name] = opt.best_estimator_
    best_params_df = pd.concat([best_params_df, pd.DataFrame([{"Model": model_name, **opt.best_params_}])])

print("\n Optimizing Deep Learning model...")
deep_opt = BayesSearchCV(
    net,
    deep_search_space,
    n_iter=250,
    cv=5,
    scoring='f1_macro',
    n_jobs=1,
    random_state=42,
    optimizer_kwargs={'n_initial_points':50}
)
deep_opt.fit(X_train_scaled.astype(np.float32), y_train)
trained_models["ANN"] = deep_opt.best_estimator_
best_params_df = pd.concat([best_params_df, pd.DataFrame([{"Model": "ANN", **deep_opt.best_params_}])])


Generating Ensemble Models

In [ ]:
model_names_list = list(val_probs.keys())
val_probs_df = pd.DataFrame({m: val_probs[m] for m in model_names_list})

# Calculate mean ensemble
ensemble_mean = val_probs_df.mean(axis=1)
val_probs["Ensemble (Mean)"] = ensemble_mean

# Calculate max ensemble
ensemble_max = val_probs_df.max(axis=1)
val_probs["Ensemble (Max)"] = ensemble_max

# Final DataFrame with everything
val_probs_df_full = pd.DataFrame(val_probs)


Model Evaluation

In [ ]:
best_params_df

In [ ]:
best_params_df.to_csv("./hp_best_params.csv")